# Opioid Crisis - Analysis

I want to take a second look at the data from the Opioid Crisis datasheet.

- (link here: https://www.mathmodels.org/Problems/2019/MCM-C/index.html)

Motivation: worked on it during 2019 for two days. Still interested, 

Data we will work with:
* Drug identification counts in years 2010-2017
* Socio-economic factors collected for five states (Ohio, Kentucky, West Virginia, Virginia, Pennsylvania)

In [1]:
import numpy as np
import pandas as pd

# drug use data.
df_nflis = pd.read_excel('2018_MCMProblemC_DATA/MCM_NFLIS_Data.xlsx', sheet_name="Data")

# socio-economic data.
df10 = pd.read_csv('2018_MCMProblemC_DATA/ACS_10_5YR_DP02/ACS_10_5YR_DP02_with_ann.csv')
df11 = pd.read_csv('2018_MCMProblemC_DATA/ACS_11_5YR_DP02/ACS_11_5YR_DP02_with_ann.csv')
# df_acs_12 = pd.read_csv('2018_MCMProblemC_DATA/ACS_12_5YR_DP02/ACS_12_5YR_DP02_with_ann.csv')
# df_acs_13 = pd.read_csv('2018_MCMProblemC_DATA/ACS_13_5YR_DP02/ACS_13_5YR_DP02_with_ann.csv')
df14 = pd.read_csv('2018_MCMProblemC_DATA/ACS_14_5YR_DP02/ACS_14_5YR_DP02_with_ann.csv')
df15 = pd.read_csv('2018_MCMProblemC_DATA/ACS_15_5YR_DP02/ACS_15_5YR_DP02_with_ann.csv')
df16 = pd.read_csv('2018_MCMProblemC_DATA/ACS_16_5YR_DP02/ACS_16_5YR_DP02_with_ann.csv')

# indexing data.
df10_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_10_5YR_DP02/ACS_10_5YR_DP02_metadata.csv')
df11_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_11_5YR_DP02/ACS_11_5YR_DP02_metadata.csv')
df14_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_14_5YR_DP02/ACS_14_5YR_DP02_metadata.csv')
df15_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_15_5YR_DP02/ACS_15_5YR_DP02_metadata.csv')
df16_meta = pd.read_csv('2018_MCMProblemC_DATA/ACS_16_5YR_DP02/ACS_16_5YR_DP02_metadata.csv')

Parts of the data are not available.

## Preprocessing

The plan is to make a `SamplePoint` class.

Feature extraction part:
* Include geography (specifically `GEO.display-label`).
* Exclude margin of error.
* Exclude columns with `(X)`.
* Exclude non-universal data.

In [2]:
df10

,GEO.id,GEO.id2,GEO.display-label,HC01_VC03,HC02_VC03,HC03_VC03,HC04_VC03,HC01_VC04,HC02_VC04,HC03_VC04,...,HC03_VC207,HC04_VC207,HC01_VC208,HC02_VC208,HC03_VC208,HC04_VC208,HC01_VC209,HC02_VC209,HC03_VC209,HC04_VC209
0,Id,Id2,Geography,Estimate; HOUSEHOLDS BY TYPE - Total households,Estimate Margin of Error; HOUSEHOLDS BY TYPE -...,Percent; HOUSEHOLDS BY TYPE - Total households,Percent Margin of Error; HOUSEHOLDS BY TYPE - ...,Estimate; HOUSEHOLDS BY TYPE - Family househol...,Estimate Margin of Error; HOUSEHOLDS BY TYPE -...,Percent; HOUSEHOLDS BY TYPE - Family household...,...,Percent; ANCESTRY - Ukrainian,Percent Margin of Error; ANCESTRY - Ukrainian,Estimate; ANCESTRY - Welsh,Estimate Margin of Error; ANCESTRY - Welsh,Percent; ANCESTRY - Welsh,Percent Margin of Error; ANCESTRY - Welsh,Estimate; ANCESTRY - West Indian (excluding Hi...,Estimate Margin of Error; ANCESTRY - West Indi...,Percent; ANCESTRY - West Indian (excluding His...,Percent Margin of Error; ANCESTRY - West India...
1,0500000US21001,21001,"Adair County, Kentucky",7272,270,7272,(X),5004,262,68.8,...,0.0,0.2,199,139,1.1,0.8,7,11,0.0,0.1
2,0500000US21003,21003,"Allen County, Kentucky",7811,291,7811,(X),5526,331,70.7,...,0.0,0.2,12,18,0.1,0.1,0,123,0.0,0.2
3,0500000US21005,21005,"Anderson County, Kentucky",8319,205,8319,(X),6148,302,73.9,...,1.0,1.3,137,98,0.6,0.5,0,123,0.0,0.2
4,0500000US21007,21007,"Ballard County, Kentucky",3363,146,3363,(X),2408,144,71.6,...,0.0,0.4,35,24,0.4,0.3,14,19,0.2,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,0500000US54101,54101,"Webster County, West Virginia",4116,214,4116,(X),2927,236,71.1,...,0.1,0.2,4,8,0.0,0.1,0,119,0.0,0.4
461,0500000US54103,54103,"Wetzel County, West Virginia",6897,223,6897,(X),4725,239,68.5,...,0.0,0.2,173,91,1.0,0.5,0,119,0.0,0.2
462,0500000US54105,54105,"Wirt County, West Virginia",2364,170,2364,(X),1697,211,71.8,...,0.0,0.6,15,23,0.3,0.4,27,30,0.5,0.5
463,0500000US54107,54107,"Wood County, West Virginia",36222,475,36222,(X),23934,562,66.1,...,0.1,0.1,569,169,0.7,0.2,0,119,0.0,0.1


The function `feature_extract` will extract features as to satisfy the above conditions (save universality).

In [3]:
from opioid_crisis_lib import feature_extract
df10[feature_extract(df10, df10_meta)]

,GEO.display-label,HC01_VC03,HC01_VC04,HC01_VC06,HC01_VC07,HC01_VC08,HC01_VC09,HC01_VC10,HC01_VC11,HC01_VC12,...,HC01_VC200,HC01_VC201,HC01_VC202,HC01_VC203,HC01_VC204,HC01_VC205,HC01_VC206,HC01_VC207,HC01_VC208,HC01_VC209
0,Geography,Estimate; HOUSEHOLDS BY TYPE - Total households,Estimate; HOUSEHOLDS BY TYPE - Family househol...,Estimate; HOUSEHOLDS BY TYPE - Family househol...,Estimate; HOUSEHOLDS BY TYPE - Family househol...,Estimate; HOUSEHOLDS BY TYPE - Family househol...,Estimate; HOUSEHOLDS BY TYPE - Family househol...,Estimate; HOUSEHOLDS BY TYPE - Family househol...,Estimate; HOUSEHOLDS BY TYPE - Family househol...,Estimate; HOUSEHOLDS BY TYPE - Family househol...,...,Estimate; ANCESTRY - Russian,Estimate; ANCESTRY - Scotch-Irish,Estimate; ANCESTRY - Scottish,Estimate; ANCESTRY - Slovak,Estimate; ANCESTRY - Subsaharan African,Estimate; ANCESTRY - Swedish,Estimate; ANCESTRY - Swiss,Estimate; ANCESTRY - Ukrainian,Estimate; ANCESTRY - Welsh,Estimate; ANCESTRY - West Indian (excluding Hi...
1,"Adair County, Kentucky",7272,5004,1945,4124,1506,377,160,503,279,...,0,370,81,0,69,20,2,0,199,7
2,"Allen County, Kentucky",7811,5526,2363,4011,1519,476,222,1039,622,...,0,350,335,24,0,0,0,0,12,0
3,"Anderson County, Kentucky",8319,6148,2771,4848,1891,267,168,1033,712,...,148,671,438,0,29,84,74,204,137,0
4,"Ballard County, Kentucky",3363,2408,1001,2001,725,193,137,214,139,...,17,253,83,8,11,101,0,0,35,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,"Webster County, West Virginia",4116,2927,1189,2365,884,216,123,346,182,...,40,325,193,0,0,45,0,9,4,0
461,"Wetzel County, West Virginia",6897,4725,1728,3845,1275,198,104,682,349,...,63,280,351,61,0,147,114,0,173,0
462,"Wirt County, West Virginia",2364,1697,601,1447,441,64,31,186,129,...,0,60,179,0,0,0,0,0,15,27
463,"Wood County, West Virginia",36222,23934,9622,18641,6666,1400,788,3893,2168,...,223,2101,1965,235,192,248,347,72,569,0


As we see only the "important" data remain. We've gotten rid of error estimate data as well as inadmissible data.

In [4]:
from opioid_crisis_lib import feature_index

ddf = [df10, df11, df14, df15, df16]
ddf_metadata = [df10_meta, df11_meta, df14_meta, df15_meta, df16_meta]

f_index = feature_index(ddf, ddf_metadata)

sorted(f_index)

['Estimate; ANCESTRY - Total population',
 'Estimate; EDUCATIONAL ATTAINMENT - Population 25 years and over',
 'Estimate; FERTILITY - Number of women 15 to 50 years old who had a birth in the past 12 months',
 'Estimate; GRANDPARENTS - Number of grandparents living with own grandchildren under 18 years',
 'Estimate; GRANDPARENTS - Number of grandparents responsible for own grandchildren under 18 years',
 'Estimate; HOUSEHOLDS BY TYPE - Households with one or more people 65 years and over',
 'Estimate; HOUSEHOLDS BY TYPE - Households with one or more people under 18 years',
 'Estimate; HOUSEHOLDS BY TYPE - Total households',
 'Estimate; LANGUAGE SPOKEN AT HOME - Population 5 years and over',
 'Estimate; MARITAL STATUS - Females 15 years and over',
 'Estimate; MARITAL STATUS - Males 15 years and over',
 'Estimate; PLACE OF BIRTH - Total population',
 'Estimate; RELATIONSHIP - Population in households',
 'Estimate; RESIDENCE 1 YEAR AGO - Population 1 year and over',
 'Estimate; SCHOOL ENR

In [5]:
f_index['Estimate; ANCESTRY - Total population']

['HC01_VC182', 'HC01_VC182', 'HC01_VC185', 'HC01_VC185', 'HC01_VC185']

We obtain a map from descriptors to corresponding labels. This allows us to go back and forth between description and label.

The above data corresponds to the socio-economic data of a particular county at some specified year. When we go over drug use data, the `YYYY`, `State` and `County` data should sufficiently return the appropriate socio-economic data.

In [6]:
from opioid_crisis_lib import state_and_county
state_and_county("Adair County, Kentucky")

('ky', 'Adair')

### Retrieving Geographic Data

In addition, for each county there should be a method to retrieve numerical geographic data:

In [13]:
df_geo = pd.read_csv('2021_Gaz_counties_national.txt', sep="\t")
from opioid_crisis_lib import locate
locate("ky", "adair", df_geo)

(37.105559, -85.28138)

In [26]:
np.array(df_geo[["USPS", "NAME"]])

array([['AL', 'Autauga County'],
       ['AL', 'Baldwin County'],
       ['AL', 'Barbour County'],
       ...,
       ['PR', 'Villalba Municipio'],
       ['PR', 'Yabucoa Municipio'],
       ['PR', 'Yauco Municipio']], dtype=object)

Index dataframes with their corresponding year.

In [8]:
ddf_yyyy = {
    "2010": df10,
    "2011": df11,
    "2014": df14,
    "2015": df15,
    "2016": df16,
}

ddf_metadata_yyyy = {
    "2010": df10_meta,
    "2011": df11_meta,
    "2014": df14_meta,
    "2015": df15_meta,
    "2016": df16_meta,
}

In [9]:
indexes = [i for i, cty in enumerate(df["NAME"]) if cty.lower() == "adair county"]
indexes

[790, 994, 1484, 2132]